In [1]:
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score
import pickle
from catboost import CatBoostClassifier

from util import *

In [13]:
#train_df = pd.read_csv("data/data-3m.csv")
import time
start_time = time.time()
test_df = pd.read_csv("data/data-202008.csv")
print("--- %s seconds ---" % (time.time() - start_time))

--- 36.619466066360474 seconds ---


In [3]:
test_df.memory_usage().sum()/ 1024**2

1262.288314819336

In [4]:
test_df["trans_amount"].memory_usage(deep=True)

57548176

In [8]:
1024*1024

1048576

In [5]:
import gc
from tqdm import tqdm

def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            print(col_type)
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                    df[col] = df[col].astype(np.uint8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                    df[col] = df[col].astype(np.uint16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                    df[col] = df[col].astype(np.uint32)                    
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
                elif c_min > np.iinfo(np.uint64).min and c_max < np.iinfo(np.uint64).max:
                    df[col] = df[col].astype(np.uint64)
            elif str(col_type)[:5] == 'float':
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
test_df = reduce_mem_usage(test_df)

Memory usage of dataframe is 1262.29 MB
float64
float64
float64
int64
float64
float64
float64
int64
Memory usage after optimization is: 1001.60 MB
Decreased by 20.7%


In [9]:
test_df.memory_usage().sum()

1050252004

In [14]:
import time
start_time = time.time()
test_df_pq = pd.read_parquet("data/data-202008.snappy.parquet")
print("--- %s seconds ---" % (time.time() - start_time))

--- 13.471753120422363 seconds ---


In [ ]:
test_df_pq.memory_usage().sum()/ 1024**2

In [15]:
gc.collect()

44